In [1]:
from pathlib import Path
USE_COLAB: bool = True
dataset_base_path = Path("/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Datasets/FakeFaces/LSGAN")
if USE_COLAB:
  from google.colab import drive
  
  # Mount the drive to access google shared docs
  drive.mount('/content/drive/', force_remount=True)

  if dataset_base_path.exists():
    print("Folder exists")
  else:
    print("DOESN'T EXIST. Add desired folder as a shortcut in your 'My Drive'")

Mounted at /content/drive/
Folder exists


In [2]:
import numpy as np
import math
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

from zipfile import ZipFile
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class Generator(nn.Module):
  def __init__(self, img_size: int = 64, latent_dim: int = 100, n_channels: int = 3):
    super(Generator, self).__init__()

    self.init_size = img_size // 4
    self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

    self.conv_blocks = nn.Sequential(
        nn.Upsample(scale_factor=2),
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(128, 0.8),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Upsample(scale_factor=2),
        nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(64, 0.8),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(in_channels=64, out_channels=n_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.Tanh(),
    )

  def forward(self, z):
    out = self.l1(z)
    out = out.view(out.shape[0], 128, self.init_size, self.init_size)
    img = self.conv_blocks(out)

    return img

In [4]:
import re
def get_latest_pth_model(base_path) -> Path:
  epoch_num = []
  all_files = sorted(Path(base_path).glob("*.pth"))
  for file_ in all_files:
    idx_num = re.search("--", str(file_)).span()
    idx_pt = re.search(".pt", str(file_)).span()
    epoch_num.append(int(str(file_)[idx_num[-1]:idx_pt[0]]))

  idx = epoch_num.index(np.max(epoch_num))
  return all_files[idx]

In [6]:
model_dir = Path("/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Code/DatasetGeneration/LSGAN/models/models")
model_path = get_latest_pth_model(model_dir)
print(f"model_path: '{model_path}'")

config = {
    "img_size": 64,
    "latent_dim": 100,
    "n_imgs_to_generate": 40000,
    "batch_size": 1,
    "n_channels": 3,
}
generator = Generator(config["img_size"], config["latent_dim"], config["n_channels"]).to(device)
checkpoint = torch.load(str(model_path))
generator.load_state_dict(checkpoint["Generator"])
generator.to(device)

model_path: '/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Code/DatasetGeneration/LSGAN/models/models/LSGAN--50.pth'


Generator(
  (l1): Sequential(
    (0): Linear(in_features=100, out_features=32768, bias=True)
  )
  (conv_blocks): Sequential(
    (0): Upsample(scale_factor=2.0, mode=nearest)
    (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Upsample(scale_factor=2.0, mode=nearest)
    (5): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Tanh()
  )
)

In [7]:
output_imgs_path = dataset_base_path
import shutil
if output_imgs_path.exists():
  shutil.rmtree(str(output_imgs_path))
output_imgs_path.mkdir(exist_ok=True, parents=True)
print(output_imgs_path)

/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Datasets/FakeFaces/LSGAN


In [8]:
from torchvision.utils import save_image
from tqdm import tqdm

torch.manual_seed(999)
if config["n_imgs_to_generate"] % config["batch_size"] != 0:
  raise RuntimeError(f"n_imgs_to_generate not divisible by batch_size!")
iterations = int(config["n_imgs_to_generate"] / config["batch_size"])
img_cnt = 0
generator.eval()
for _ in tqdm(range(iterations)):
  with torch.no_grad():
    fakes = generator(torch.randn(config["batch_size"], config["latent_dim"], dtype=torch.float, device=device))
    for fake in fakes:
      output_path = output_imgs_path / f"{img_cnt}.jpg"
      save_image(fake, output_path)
      img_cnt += 1
    
drive.flush_and_unmount()

100%|██████████| 40000/40000 [04:38<00:00, 143.72it/s]
